In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

train = pd.read_csv("/kaggle/input/summeranalytics/hacktrain.csv")
test = pd.read_csv("/kaggle/input/summeranalytics/hacktest.csv")

train.fillna(train.mean(numeric_only=True), inplace=True)
test.fillna(test.mean(numeric_only=True), inplace=True)

test_ids = test['ID']
train.drop(columns=['ID'], inplace=True)
test.drop(columns=['ID'], inplace=True)

label_encoder = LabelEncoder()
train['class'] = label_encoder.fit_transform(train['class'])

X = train.drop(columns=['class'])
y = train['class']

X['ndvi_mean'] = X.mean(axis=1)
X['ndvi_std'] = X.std(axis=1)
X['ndvi_min'] = X.min(axis=1)
X['ndvi_max'] = X.max(axis=1)

test['ndvi_mean'] = test.mean(axis=1)
test['ndvi_std'] = test.std(axis=1)
test['ndvi_min'] = test.min(axis=1)
test['ndvi_max'] = test.max(axis=1)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=700,
    C=2.0
)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=label_encoder.classes_))

test_preds = model.predict(test)
decoded_preds = label_encoder.inverse_transform(test_preds)

submission = pd.DataFrame({
    "ID": test_ids,
    "class": decoded_preds
})
submission.to_csv("SummerAnalytics.csv", index=False)
print("submission.csv created.")

              precision    recall  f1-score   support

        farm       0.83      0.84      0.84       168
      forest       0.99      0.99      0.99      1232
       grass       0.89      0.62      0.73        39
  impervious       0.81      0.87      0.84       134
     orchard       0.33      0.33      0.33         6
       water       0.85      0.81      0.83        21

    accuracy                           0.95      1600
   macro avg       0.78      0.74      0.76      1600
weighted avg       0.95      0.95      0.95      1600

submission.csv created.


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
